In [115]:
import pandas as pd
import numpy as np
import os


import keras
from keras.layers import Input, Dense, BatchNormalization, Activation, Dropout
from keras.optimizers import Adam
from keras.models import Model

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [2]:
# Init params and helpers

In [32]:
model_tag = '01'
data_dir = '../data/'
cleaned_csv_file_name = 'cleaned.csv'
models_dir = './models'
logs_dir = './logs/'

num_k = 10

In [36]:
kfold_data_dir = '../data/k-fold/'
file_list = os.listdir(kfold_data_dir)
print(len(file_list))

num_k = int(len(file_list) / 2)
print(f'Number of K is {num_k}')

file_train_start = 'train'
file_test_start = 'test'


20
Number of K is 10


In [4]:
# define features and classes

In [5]:
predict_class_right = ['PostOp-SNR',  'PostOp-CNR',  'PostOp-BWR', 'PostOp-BHR',
                       'PostOp-IMFR',  'PostOp-AreRW', 'PostOp-AreRH']

predict_class_left = [ 'PostOp-SNL',  'PostOp-CNL', 'PostOp-BWL',
                      'PostOp-BHL',  'PostOp-IMFL','PostOp-AreLW', 'PostOp-AreLH']

predict_post_id = ['PostOp-ID']

feature_left = ['PreOp-SNL', 'PreOp-CNL', 'PreOp-BWL', 'PreOp-BHL', 'PreOp-IMFL', 'PreOp-AreLW',
                'PreOp-AreLH', 'PreOp-ID', 'ImplantLeft']

feature_right = ['PreOp-SNR', 'PreOp-CNR', 'PreOp-BWR', 'PreOp-BHR', 'PreOp-IMFR', 'PreOp-AreRW',
                'PreOp-AreRH', 'PreOp-ID', 'ImplantRight']

In [6]:
# Check diractory

In [14]:
if not os.path.exists(os.path.join(models_dir, model_tag)):
    os.mkdir(os.path.join(models_dir, model_tag))
    
if not os.path.exists(os.path.join(logs_dir, model_tag)):
    os.mkdir(os.path.join(logs_dir, model_tag))
    
for predict_class in predict_class_right+predict_class_left+predict_post_id:
    if not os.path.exists(os.path.join(models_dir, model_tag, predict_class)):
        os.mkdir(os.path.join(models_dir, model_tag, predict_class))
        
for predict_class in predict_class_right+predict_class_left+predict_post_id:
    if not os.path.exists(os.path.join(logs_dir, model_tag, predict_class)):
        os.mkdir(os.path.join(logs_dir, model_tag, predict_class))

In [15]:
def get_x_and_y(df: pd.DataFrame, x_head=[], y_head=[]):
    x, y = [], []
    
    for index, row in df.iterrows():
        x_temp = []
        for x_h in x_head:
            x_temp.append(row[x_h])
        x.append(x_temp)
        y.append(row[y_head[0]])
    
    return x, y

In [112]:
def create_model(feature_len):
    inp = Input(shape=(feature_len,))
    x = Dropout(0.25)(inp)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    output = Dense(1, activation='linear')(x)
    model = Model(inp, output)
    
    loss = 'mse'
    
    model.compile(optimizer=Adam(1e-4),
                  loss=loss)
    return model

In [113]:
def get_callbacks(prediction_tag):
    checkpoint_path = os.path.join()
    
    
    er = EarlyStopping(monitor="val_loss", mode="min", patience=20)
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_categorical_accuracy', verbose=1, save_best_only=True,  mode='max')

In [111]:
def get_accuracy(model, x, y):
    result = model.predict(x)
    sum = 0
    for r, y in zip(result, y):
        sum += abs(r - y)
    return (sum / len(x))[0]

In [72]:
accuracies = []

prediction_tag = 'PostOp-SNL'
for k in range(num_k):
    train_df = pd.read_csv(os.path.join(kfold_data_dir, f'{file_train_start}_{k}.csv',)).dropna()
    test_df = pd.read_csv(os.path.join(kfold_data_dir, f'{file_test_start}_{k}.csv')).dropna()
    
    # Create right side model
    
    x_train, y_train = get_x_and_y(train_df, feature_left, [prediction_tag])
    x_test, y_test = get_x_and_y(test_df, feature_left, [prediction_tag])
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    
    model = create_model(len(x_train[0]))
    history = model.fit(x_train, y_train,
                        batch_size=8,
                        epochs=1000,
                        callbacks=get_callbacks(prediction_tag),
                        validation_data=[x_test, y_test])
    accuracies.append(x_test)
    break
    model.fit(x_train, y_train)
    k_result = model.predict(x_test)
    result.append(sum(abs(k_result - y_test)) / len(k_result))

# print(sum(result) / len(result))

Train on 92 samples, validate on 8 samples
Epoch 1/1000
92/92 [==============================] - 2s 27ms/step - loss: 328.9604 - val_loss: 289.3433
Epoch 2/1000
92/92 [==============================] - 0s 1ms/step - loss: 321.5155 - val_loss: 284.0746
Epoch 3/1000
92/92 [==============================] - 0s 1ms/step - loss: 315.2375 - val_loss: 279.4064
Epoch 4/1000
92/92 [==============================] - 0s 1ms/step - loss: 307.2240 - val_loss: 276.2655
Epoch 5/1000
92/92 [==============================] - 0s 1ms/step - loss: 302.2674 - val_loss: 270.7745
Epoch 6/1000
92/92 [==============================] - 0s 1ms/step - loss: 295.9307 - val_loss: 266.1844
Epoch 7/1000
92/92 [==============================] - 0s 1ms/step - loss: 290.9756 - val_loss: 263.8107
Epoch 8/1000
92/92 [==============================] - 0s 1ms/step - loss: 282.6711 - val_loss: 261.2180
Epoch 9/1000
92/92 [==============================] - 0s 1ms/step - loss: 279.2755 - val_loss: 253.6035
Epoch 10/1000
92/92 

92/92 [==============================] - 0s 1ms/step - loss: 55.6831 - val_loss: 20.1181
Epoch 80/1000
92/92 [==============================] - 0s 1ms/step - loss: 55.0445 - val_loss: 18.2346
Epoch 81/1000
92/92 [==============================] - 0s 1ms/step - loss: 46.9649 - val_loss: 15.2783
Epoch 82/1000
92/92 [==============================] - 0s 1ms/step - loss: 52.8945 - val_loss: 14.5632
Epoch 83/1000
92/92 [==============================] - 0s 1ms/step - loss: 53.2630 - val_loss: 15.7477
Epoch 84/1000
92/92 [==============================] - 0s 1ms/step - loss: 47.2823 - val_loss: 16.1120
Epoch 85/1000
92/92 [==============================] - 0s 1ms/step - loss: 49.2661 - val_loss: 17.9416
Epoch 86/1000
92/92 [==============================] - 0s 1ms/step - loss: 48.7532 - val_loss: 20.7937
Epoch 87/1000
92/92 [==============================] - 0s 1ms/step - loss: 43.1207 - val_loss: 19.1749
Epoch 88/1000
92/92 [==============================] - 0s 1ms/step - loss: 47.4686 - va

Epoch 159/1000
92/92 [==============================] - 0s 1ms/step - loss: 6.8614 - val_loss: 2.2938
Epoch 160/1000
92/92 [==============================] - 0s 1ms/step - loss: 6.6582 - val_loss: 2.7423
Epoch 161/1000
92/92 [==============================] - 0s 1ms/step - loss: 5.8678 - val_loss: 2.3216
Epoch 162/1000
92/92 [==============================] - 0s 1ms/step - loss: 4.8762 - val_loss: 2.3930
Epoch 163/1000
92/92 [==============================] - 0s 1ms/step - loss: 6.6068 - val_loss: 2.3707
Epoch 164/1000
92/92 [==============================] - 0s 1ms/step - loss: 5.4411 - val_loss: 2.0242
Epoch 165/1000
92/92 [==============================] - 0s 1ms/step - loss: 7.8605 - val_loss: 2.0197
Epoch 166/1000
92/92 [==============================] - 0s 1ms/step - loss: 4.8201 - val_loss: 2.7477
Epoch 167/1000
92/92 [==============================] - 0s 1ms/step - loss: 5.8386 - val_loss: 4.0110
Epoch 168/1000
92/92 [==============================] - 0s 1ms/step - loss: 5.5507

92/92 [==============================] - 0s 1ms/step - loss: 3.7031 - val_loss: 1.1334
Epoch 239/1000
92/92 [==============================] - 0s 903us/step - loss: 2.7612 - val_loss: 2.1074
Epoch 240/1000
92/92 [==============================] - 0s 870us/step - loss: 4.4352 - val_loss: 2.0919
Epoch 241/1000
92/92 [==============================] - 0s 903us/step - loss: 3.3620 - val_loss: 1.5506
Epoch 242/1000
92/92 [==============================] - 0s 914us/step - loss: 3.4282 - val_loss: 1.5161
Epoch 243/1000
92/92 [==============================] - 0s 947us/step - loss: 3.6014 - val_loss: 1.6973
Epoch 244/1000
92/92 [==============================] - 0s 925us/step - loss: 4.0820 - val_loss: 1.5416
Epoch 245/1000
92/92 [==============================] - 0s 903us/step - loss: 4.6989 - val_loss: 1.4687
Epoch 246/1000
92/92 [==============================] - 0s 936us/step - loss: 4.4601 - val_loss: 1.2580
Epoch 247/1000
92/92 [==============================] - 0s 903us/step - loss: 3.6

92/92 [==============================] - 0s 1ms/step - loss: 2.9939 - val_loss: 0.7507
Epoch 318/1000
92/92 [==============================] - 0s 1ms/step - loss: 2.6754 - val_loss: 0.8633
Epoch 319/1000
92/92 [==============================] - 0s 968us/step - loss: 3.4172 - val_loss: 0.8415
Epoch 320/1000
92/92 [==============================] - 0s 979us/step - loss: 2.7957 - val_loss: 0.7248
Epoch 321/1000
92/92 [==============================] - 0s 968us/step - loss: 1.9422 - val_loss: 0.6972
Epoch 322/1000
92/92 [==============================] - 0s 936us/step - loss: 2.7207 - val_loss: 0.8684
Epoch 323/1000
92/92 [==============================] - 0s 968us/step - loss: 3.0934 - val_loss: 0.7221
Epoch 324/1000
92/92 [==============================] - 0s 1ms/step - loss: 2.8798 - val_loss: 0.7325
Epoch 325/1000
92/92 [==============================] - 0s 892us/step - loss: 2.3618 - val_loss: 0.7342
Epoch 326/1000
92/92 [==============================] - 0s 870us/step - loss: 2.9530 

...

Epoch 633/1000
92/92 [==============================] - 0s 914us/step - loss: 2.8561 - val_loss: 0.9589
Epoch 634/1000
92/92 [==============================] - 0s 903us/step - loss: 2.1784 - val_loss: 0.9352
Epoch 635/1000
92/92 [==============================] - 0s 925us/step - loss: 1.8563 - val_loss: 0.9359
Epoch 636/1000
92/92 [==============================] - 0s 936us/step - loss: 1.6188 - val_loss: 0.9612
Epoch 637/1000
92/92 [==============================] - 0s 914us/step - loss: 2.0437 - val_loss: 0.8166
Epoch 638/1000
92/92 [==============================] - 0s 914us/step - loss: 1.4376 - val_loss: 0.9798
Epoch 639/1000
92/92 [==============================] - 0s 903us/step - loss: 2.3685 - val_loss: 1.0511
Epoch 640/1000
92/92 [==============================] - 0s 903us/step - loss: 1.8864 - val_loss: 1.2561
Epoch 641/1000
92/92 [==============================] - 0s 914us/step - loss: 1.7104 - val_loss: 1.1357
Epoch 642/1000
92/92 [==============================] - 0s 903us

Epoch 712/1000
92/92 [==============================] - 0s 947us/step - loss: 2.0537 - val_loss: 0.6334
Epoch 713/1000
92/92 [==============================] - 0s 892us/step - loss: 1.8554 - val_loss: 0.6315
Epoch 714/1000
92/92 [==============================] - 0s 903us/step - loss: 1.8904 - val_loss: 0.6047
Epoch 715/1000
92/92 [==============================] - 0s 892us/step - loss: 1.9135 - val_loss: 0.5837
Epoch 716/1000
92/92 [==============================] - 0s 892us/step - loss: 1.9236 - val_loss: 0.5931
Epoch 717/1000
92/92 [==============================] - 0s 903us/step - loss: 1.6452 - val_loss: 0.5411
Epoch 718/1000
92/92 [==============================] - 0s 903us/step - loss: 1.9097 - val_loss: 0.6160
Epoch 719/1000
92/92 [==============================] - 0s 947us/step - loss: 1.9584 - val_loss: 0.6556
Epoch 720/1000
92/92 [==============================] - 0s 979us/step - loss: 2.0304 - val_loss: 0.7739
Epoch 721/1000
92/92 [==============================] - 0s 892us

Epoch 791/1000
92/92 [==============================] - 0s 903us/step - loss: 2.2601 - val_loss: 0.8508
Epoch 792/1000
92/92 [==============================] - 0s 903us/step - loss: 1.5026 - val_loss: 0.8361
Epoch 793/1000
92/92 [==============================] - 0s 892us/step - loss: 1.3835 - val_loss: 0.8211
Epoch 794/1000
92/92 [==============================] - 0s 903us/step - loss: 1.7333 - val_loss: 0.8224
Epoch 795/1000
92/92 [==============================] - 0s 925us/step - loss: 2.1790 - val_loss: 0.8748
Epoch 796/1000
92/92 [==============================] - 0s 903us/step - loss: 1.5674 - val_loss: 0.9204
Epoch 797/1000
92/92 [==============================] - 0s 903us/step - loss: 1.5828 - val_loss: 0.9446
Epoch 798/1000
92/92 [==============================] - 0s 925us/step - loss: 2.0005 - val_loss: 0.7842
Epoch 799/1000
92/92 [==============================] - 0s 936us/step - loss: 2.0910 - val_loss: 0.8525
Epoch 800/1000
92/92 [==============================] - 0s 947us

Epoch 870/1000
92/92 [==============================] - 0s 914us/step - loss: 2.2916 - val_loss: 0.7291
Epoch 871/1000
92/92 [==============================] - 0s 903us/step - loss: 2.6513 - val_loss: 0.7049
Epoch 872/1000
92/92 [==============================] - 0s 903us/step - loss: 1.5637 - val_loss: 0.7338
Epoch 873/1000
92/92 [==============================] - 0s 914us/step - loss: 2.2156 - val_loss: 0.7141
Epoch 874/1000
92/92 [==============================] - 0s 957us/step - loss: 1.7479 - val_loss: 0.6534
Epoch 875/1000
92/92 [==============================] - 0s 925us/step - loss: 1.5302 - val_loss: 0.7026
Epoch 876/1000
92/92 [==============================] - 0s 914us/step - loss: 1.5344 - val_loss: 0.7436
Epoch 877/1000
92/92 [==============================] - 0s 903us/step - loss: 1.8623 - val_loss: 0.7385
Epoch 878/1000
92/92 [==============================] - 0s 936us/step - loss: 1.8100 - val_loss: 0.7377
Epoch 879/1000
92/92 [==============================] - 0s 914us

Epoch 949/1000
92/92 [==============================] - 0s 859us/step - loss: 1.8207 - val_loss: 1.1782
Epoch 950/1000
92/92 [==============================] - 0s 870us/step - loss: 2.3538 - val_loss: 1.2508
Epoch 951/1000
92/92 [==============================] - 0s 892us/step - loss: 1.7394 - val_loss: 1.2599
Epoch 952/1000
92/92 [==============================] - 0s 892us/step - loss: 1.8455 - val_loss: 1.2552
Epoch 953/1000
92/92 [==============================] - 0s 849us/step - loss: 2.3022 - val_loss: 1.1320
Epoch 954/1000
92/92 [==============================] - 0s 870us/step - loss: 1.2440 - val_loss: 1.0659
Epoch 955/1000
92/92 [==============================] - 0s 870us/step - loss: 2.1022 - val_loss: 1.2238
Epoch 956/1000
92/92 [==============================] - 0s 892us/step - loss: 1.6371 - val_loss: 1.2951
Epoch 957/1000
92/92 [==============================] - 0s 892us/step - loss: 1.4305 - val_loss: 1.3861
Epoch 958/1000
92/92 [==============================] - 0s 870us

1.1320543